# Fine-tuning BERT Spanish

In [1]:
# !pip install -q transformers datasets

## Load dataset

Next, let's download a multi-label text classification dataset [MultiEURLEX](https://huggingface.co/datasets/multi_eurlex)


In [5]:
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

In [7]:
from datasets import load_dataset

dataset = load_dataset('multi_eurlex', 'es')

Generating train split:   0%|          | 0/52785 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/5000 [00:00<?, ? examples/s]

Dataset multi_eurlex downloaded and prepared to /home/davo/.cache/huggingface/datasets/multi_eurlex/es/1.0.0/8ec8b79877a517369a143ead6679d1788d13e51cf641ed29772f4449e8364fb6. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

As we can see, the dataset contains 3 splits: one for training, one for validation and one for testing.

In [8]:

dataset = dataset.rename_column("labels", "old_labels")

In [9]:

dataset

DatasetDict({
    train: Dataset({
        features: ['celex_id', 'text', 'old_labels'],
        num_rows: 52785
    })
    test: Dataset({
        features: ['celex_id', 'text', 'old_labels'],
        num_rows: 5000
    })
    validation: Dataset({
        features: ['celex_id', 'text', 'old_labels'],
        num_rows: 5000
    })
})

Let's check the first example of the training split:

In [10]:
example = dataset['train'][0]
example

{'celex_id': '32006D0213',
 'text': 'DECISIÓN DE LA COMISIÓN\nde 6 de marzo de 2006\npor la que se establecen las clases de reacción al fuego para determinados productos de construcción en lo que respecta a los suelos de madera y los paneles y revestimientos murales de madera maciza\n[notificada con el número C(2006) 655]\n(Texto pertinente a efectos del EEE)\n(2006/213/CE)\nLA COMISIÓN DE LAS COMUNIDADES EUROPEAS,\nVisto el Tratado constitutivo de la Comunidad Europea,\nVista la Directiva 89/106/CEE del Consejo, de 21 de diciembre de 1988, relativa a la aproximación de las disposiciones legales, reglamentarias y administrativas de los Estados miembros sobre los productos de construcción (1), y, en particular, su artículo 20, apartado 2,\nConsiderando lo siguiente:\n(1)\nEn la Directiva 89/106/CEE se establece que, con el fin de tener en cuenta los distintos niveles de protección para las obras de construcción a escala nacional, regional o local, puede ser necesario establecer, en los 

The dataset consists of tweets, labeled with one or more emotions. 

Let's create a list that contains the labels, as well as 2 dictionaries that map labels to integers and back.

In [11]:
labels = ['social questions',
 'industry',
 'finance',
 'trade',
 'business and competition',
 'international relations',
 'agriculture, forestry and fisheries',
 'production, technology and research',
 'transport',
 'employment and working conditions',
 'politics',
 'law',
 'education and communications',
 'international organisations',
 'energy',
 'EUROPEAN UNION',
 'science',
 'agri-foodstuffs',
 'geography',
 'economics',
 'environment']
id2label = {0: 'social questions',
 1: 'industry',
 2: 'finance',
 3: 'trade',
 4: 'business and competition',
 5: 'international relations',
 6: 'agriculture, forestry and fisheries',
 7: 'production, technology and research',
 8: 'transport',
 9: 'employment and working conditions',
 10: 'politics',
 11: 'law',
 12: 'education and communications',
 13: 'international organisations',
 14: 'energy',
 15: 'EUROPEAN UNION',
 16: 'science',
 17: 'agri-foodstuffs',
 18: 'geography',
 19: 'economics',
 20: 'environment'}
label2id = {'social questions': 0,
 'industry': 1,
 'finance': 2,
 'trade': 3,
 'business and competition': 4,
 'international relations': 5,
 'agriculture, forestry and fisheries': 6,
 'production, technology and research': 7,
 'transport': 8,
 'employment and working conditions': 9,
 'politics': 10,
 'law': 11,
 'education and communications': 12,
 'international organisations': 13,
 'energy': 14,
 'EUROPEAN UNION': 15,
 'science': 16,
 'agri-foodstuffs': 17,
 'geography': 18,
 'economics': 19,
 'environment': 20}
labels

['social questions',
 'industry',
 'finance',
 'trade',
 'business and competition',
 'international relations',
 'agriculture, forestry and fisheries',
 'production, technology and research',
 'transport',
 'employment and working conditions',
 'politics',
 'law',
 'education and communications',
 'international organisations',
 'energy',
 'EUROPEAN UNION',
 'science',
 'agri-foodstuffs',
 'geography',
 'economics',
 'environment']

## Preprocess data

As models like BERT don't expect text as direct input, but rather `input_ids`, etc., we tokenize the text using the tokenizer. Here I'm using the `AutoTokenizer` API, which will automatically load the appropriate tokenizer based on the checkpoint on the hub.

What's a bit tricky is that we also need to provide labels to the model. For multi-label text classification, this is a matrix of shape (batch_size, num_labels). Also important: this should be a tensor of floats rather than integers, otherwise PyTorch' `BCEWithLogitsLoss` (which the model will use) will complain, as explained [here](https://discuss.pytorch.org/t/multi-label-binary-classification-result-type-float-cant-be-cast-to-the-desired-output-type-long/117915/3).

In [12]:
def numbers_to_classes(l=[0,10,20]):
  zero_cl = [0.0] * 21
  for i in l:
    zero_cl[i] = 1.0
  
  return np.array(zero_cl, dtype=np.float)

In [13]:
numbers_to_classes(l=[0,10,20])

array([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 1.])

In [14]:
from transformers import AutoTokenizer
import numpy as np
import pandas as pd

tokenizer = AutoTokenizer.from_pretrained("bert-base-multilingual-uncased")

def preprocess_data(examples):
  # take a batch of texts
  text = examples["text"]
  # encode them
  encoding = tokenizer(text, padding="max_length", truncation=True, max_length=300)
  # add labels
  labels_matrix = []
  for r in examples["old_labels"]:
    labels_matrix.append(np.array(numbers_to_classes(r), dtype=float))
  # print(labels_matrix)
  encoding["labels"] = np.array(labels_matrix, dtype=float)
  
  return encoding

In [15]:
encoded_dataset = dataset.map(preprocess_data, batched=True)

Parameter 'function'=<function preprocess_data at 0x7fed8f7fd8c0> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/53 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [16]:
example = encoded_dataset['train'][0]
print(example.keys())

dict_keys(['celex_id', 'text', 'old_labels', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'])


In [17]:
tokenizer.decode(example['input_ids'])

'[CLS] decision de la comision de 6 de marzo de 2006 por la que se establecen las clases de reaccion al fuego para determinados productos de construccion en lo que respecta a los suelos de madera y los paneles y revestimientos murales de madera maciza [ notificada con el numero c ( 2006 ) 655 ] ( texto pertinente a efectos del eee ) ( 2006 / 213 / ce ) la comision de las comunidades europeas, visto el tratado constitutivo de la comunidad europea, vista la directiva 89 / 106 / cee del consejo, de 21 de diciembre de 1988, relativa a la aproximacion de las disposiciones legales, reglamentarias y administrativas de los estados miembros sobre los productos de construccion ( 1 ), y, en particular, su articulo 20, apartado 2, considerando lo siguiente : ( 1 ) en la directiva 89 / 106 / cee se establece que, con el fin de tener en cuenta los distintos niveles de proteccion para las obras de construccion a escala nacional, regional o local, puede ser necesario establecer, en los documentos inte

In [18]:
[id2label[idx] for idx, label in enumerate(example['labels']) if label == 1.0]

['social questions',
 'industry',
 'trade',
 'production, technology and research',
 'environment']

Finally, we set the format of our data to PyTorch tensors. This will turn the training, validation and test sets into standard PyTorch [datasets](https://pytorch.org/docs/stable/data.html). 

In [19]:
encoded_dataset.set_format("torch")

## Define model

Here we define a model that includes a pre-trained base (i.e. the weights from bert-base-multilingual-uncased) are loaded, with a random initialized classification head (linear layer) on top. One should fine-tune this head, together with the pre-trained base on a labeled dataset.

This is also printed by the warning.

We set the `problem_type` to be "multi_label_classification", as this will make sure the appropriate loss function is used (namely [`BCEWithLogitsLoss`](https://pytorch.org/docs/stable/generated/torch.nn.BCEWithLogitsLoss.html)). We also make sure the output layer has `len(labels)` output neurons, and we set the id2label and label2id mappings.

In [20]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-multilingual-uncased", 
                                                           problem_type="multi_label_classification", 
                                                           num_labels=len(labels),
                                                           id2label=id2label,
                                                           label2id=label2id)

Some weights of the model checkpoint at bert-base-multilingual-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model 

## Train the model!

We are going to train the model using HuggingFace's Trainer API. This requires us to define 2 things: 

* `TrainingArguments`, which specify training hyperparameters. All options can be found in the [docs](https://huggingface.co/transformers/main_classes/trainer.html#trainingarguments). Below, we for example specify that we want to evaluate after every epoch of training, we would like to save the model every epoch, we set the learning rate, the batch size to use for training/evaluation, how many epochs to train for, and so on.
* a `Trainer` object (docs can be found [here](https://huggingface.co/transformers/main_classes/trainer.html#id1)).

In [21]:
batch_size = 16
metric_name = "f1"

In [28]:
from transformers import TrainingArguments, Trainer

args = TrainingArguments(
    f"bert-finetuned-sem_eval-french",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model=metric_name,
    #push_to_hub=True,
)

We are also going to compute metrics while training. For this, we need to define a `compute_metrics` function, that returns a dictionary with the desired metric values.

In [29]:
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
from transformers import EvalPrediction
import torch
    
# source: https://jesusleal.io/2021/04/21/Longformer-multilabel-classification/
def multi_label_metrics(predictions, labels, threshold=0.5):
    # first, apply sigmoid on predictions which are of shape (batch_size, num_labels)
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(predictions))
    # next, use threshold to turn them into integer predictions
    y_pred = np.zeros(probs.shape)
    y_pred[np.where(probs >= threshold)] = 1
    # finally, compute metrics
    y_true = labels
    f1_micro_average = f1_score(y_true=y_true, y_pred=y_pred, average='micro')
    roc_auc = roc_auc_score(y_true, y_pred, average = 'micro')
    accuracy = accuracy_score(y_true, y_pred)
    # return as dictionary
    metrics = {'f1': f1_micro_average,
               'roc_auc': roc_auc,
               'accuracy': accuracy}
    return metrics

def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, 
            tuple) else p.predictions
    result = multi_label_metrics(
        predictions=preds, 
        labels=p.label_ids)
    return result

Let's verify a batch as well as a forward pass:

In [30]:
encoded_dataset['train']['labels'][0]

tensor([1., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 1.])

In [31]:

encoded_dataset['train']['labels']

tensor([[1., 1., 0.,  ..., 0., 0., 1.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 1., 0.],
        ...,
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 1.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])

In [32]:
encoded_dataset['train']['input_ids'][0]

tensor([  101, 15308, 10102, 10106, 28284, 10102,   127, 10102, 12620, 10102,
        10231, 10190, 10106, 10126, 10128, 74125, 10115, 10265, 38018, 10102,
        49483, 10161, 33928, 10239, 15752, 10107, 32746, 10102, 19184, 10109,
        10387, 10126, 26497, 10112,   143, 10175, 45134, 10107, 10102, 33017,
          167, 10175, 29223, 10165,   167, 90353, 60560, 72104, 76640, 10165,
        10102, 33017, 13501, 26543,   138, 69327, 41707, 10173, 10117, 11855,
          145,   113, 10231,   114, 49168,   140,   113, 26081, 10174, 50067,
        10111,   143, 34489, 10134, 23329, 10111,   114,   113, 10231,   120,
        21232,   120, 10630,   114, 10106, 28284, 10102, 10265, 41136, 89022,
          117, 21963, 10117, 26833, 10173, 12054, 44309, 14635, 10102, 10106,
        24073, 20343,   117, 14681, 10106, 80658, 12844,   120, 16141,   120,
        10630, 10111, 10134, 25042,   117, 10102, 10259, 10102, 16409, 10102,
        10632,   117, 40840,   143, 10106, 54206, 42134, 12611, 

In [33]:
#forward pass
outputs = model(input_ids=encoded_dataset['train']['input_ids'][0].unsqueeze(0), labels=encoded_dataset['train']['labels'][0].unsqueeze(0))
outputs

SequenceClassifierOutput(loss=tensor(0.7053, grad_fn=<BinaryCrossEntropyWithLogitsBackward>), logits=tensor([[-0.0353,  0.1318,  0.0647,  0.2442,  0.1971,  0.1151,  0.0311, -0.0358,
         -0.0739, -0.0315, -0.1204,  0.0748, -0.1109, -0.2053, -0.0148, -0.0814,
          0.2163,  0.2505,  0.1540,  0.2158, -0.0332]],
       grad_fn=<AddmmBackward>), hidden_states=None, attentions=None)

Let's start training!

In [34]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset["train"],
    eval_dataset=encoded_dataset["validation"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)
/home/davo/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: greenlet.greenlet size changed, may indicate binary incompatibility. Expected 144 from C header, got 152 from PyObject
  return f(*args, **kwds)


In [35]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, celex_id, old_labels. If text, celex_id, old_labels are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
/home/davo/anaconda3/lib/python3.7/site-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 52785
  Num Epochs = 10
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 33000
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: Currently logged in as:

Epoch,Training Loss,Validation Loss,F1,Roc Auc,Accuracy
1,0.150100,0.176955,0.798811,0.855899,0.242000
2,0.125500,0.166279,0.821560,0.875524,0.291000
3,0.109800,0.165537,0.830239,0.886043,0.315200
4,0.097700,0.169398,0.828400,0.885860,0.309400
5,0.087000,0.173899,0.827146,0.885800,0.307400
6,0.076100,0.177948,0.828656,0.887494,0.320400
7,0.069000,0.181404,0.827658,0.887668,0.318800
8,0.059900,0.187455,0.825819,0.886064,0.313800
9,0.055500,0.190041,0.827078,0.886931,0.321000
10,0.050100,0.191350,0.826848,0.887791,0.319600


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, celex_id, old_labels. If text, celex_id, old_labels are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to bert-finetuned-sem_eval-french/checkpoint-3300
Configuration saved in bert-finetuned-sem_eval-french/checkpoint-3300/config.json
Model weights saved in bert-finetuned-sem_eval-french/checkpoint-3300/pytorch_model.bin
tokenizer config file saved in bert-finetuned-sem_eval-french/checkpoint-3300/tokenizer_config.json
Special tokens file saved in bert-finetuned-sem_eval-french/checkpoint-3300/special_tokens_map.json
The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, celex_id, old_labels. If text



Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from bert-finetuned-sem_eval-french/checkpoint-9900 (score: 0.8302390279906887).


TrainOutput(global_step=33000, training_loss=0.092083921548092, metrics={'train_runtime': 13214.8913, 'train_samples_per_second': 39.944, 'train_steps_per_second': 2.497, 'total_flos': 8.139074010632998e+16, 'train_loss': 0.092083921548092, 'epoch': 10.0})

## Evaluate

After training, we evaluate our model on the validation set.

In [36]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: text, celex_id, old_labels. If text, celex_id, old_labels are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16


{'eval_loss': 0.16553713381290436,
 'eval_f1': 0.8302390279906887,
 'eval_roc_auc': 0.8860434580006998,
 'eval_accuracy': 0.3152,
 'eval_runtime': 36.0575,
 'eval_samples_per_second': 138.668,
 'eval_steps_per_second': 8.681,
 'epoch': 10.0}

## Inference

Let's test the model on a new sentence:

In [37]:
dataset['test']['text'][0]

'REGLAMENTO (UE) No 1390/2013 DEL CONSEJO\nde 16 de diciembre de 2013\nrelativo al reparto de las posibilidades de pesca en virtud del Protocolo entre la Unión Europea y la Unión de las Comoras por el que se fijan las posibilidades de pesca y la contrapartida financiera establecidas en el Acuerdo de Colaboración en el sector pesquero en vigor entre ambas Partes\nEL CONSEJO DE LA UNIÓN EUROPEA,\nVisto el Tratado de Funcionamiento de la Unión Europea, y en particular su artículo 43, apartado 3,\nVista la propuesta de la Comisión Europea,\nConsiderando lo siguiente:\n(1)\nEl 5 de octubre de 2006, el Consejo aprobó el Acuerdo de Colaboración en el sector pesquero entre la Comunidad Europea y la Unión de las Comoras («Acuerdo de Colaboración»), mediante la adopción del Reglamento (CE) no 1563/2006 (1).\n(2)\nLa Unión Europea negoció con la Unión de las Comoras un nuevo Protocolo del Acuerdo de Colaboración, por el que se conceden a los buques de la Unión Europea posibilidades de pesca en ag

### Below you can observe a few examples with their predicted outputs for Bert model and what actually was true label

In [38]:
def predict_labels(text, thres_prob=0.5):
    encoding = tokenizer(text, return_tensors="pt")
    encoding = {k: v.to(trainer.model.device) for k,v in encoding.items()}

    outputs = trainer.model(**encoding)
    logits = outputs.logits
    # apply sigmoid + threshold
    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(logits.squeeze().cpu())
    predictions = np.zeros(probs.shape)
    predictions[np.where(probs >= thres_prob)] = 1
    # turn predicted id's into actual label names
    predicted_labels = [id2label[idx] for idx, label in enumerate(predictions) if label == 1.0]

    return predicted_labels

In [39]:
predicted_labels = predict_labels(text=dataset['test']['text'][0][:512])

In [40]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['international relations', 'agriculture, forestry and fisheries', 'geography']


In [41]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][0])) if label == 1.0])

true labels>>> ['international relations', 'agriculture, forestry and fisheries', 'EUROPEAN UNION', 'geography']


### In the case above Bert model was able to pick up all true categories but missed 'EUROPEAN UNION'  category

In [42]:
predicted_labels = predict_labels(text=dataset['test']['text'][100][:512])

In [43]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['trade', 'agriculture, forestry and fisheries', 'agri-foodstuffs', 'geography']


In [44]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][100])) if label == 1.0])

true labels>>> ['trade', 'agriculture, forestry and fisheries', 'production, technology and research', 'agri-foodstuffs', 'geography']


### In the case above Bert model was able to pick up all true categories but missed 'production, technology and research'  category

In [45]:
predicted_labels = predict_labels(text=dataset['test']['text'][1000][:512])

In [46]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['international relations', 'EUROPEAN UNION', 'geography']


In [47]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][1000])) if label == 1.0])

true labels>>> ['trade', 'international relations', 'EUROPEAN UNION', 'geography']


### In the case above Bert model was able to pick up all true categories but also missed 'trade' category

In [53]:
predicted_labels = predict_labels(text=dataset['test']['text'][1000][:512], thres_prob=0.2)

In [54]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['trade', 'international relations', 'EUROPEAN UNION', 'geography']


In [55]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][1000])) if label == 1.0])

true labels>>> ['trade', 'international relations', 'EUROPEAN UNION', 'geography']


### if we reduce threshold to 20% for accepting  a label as related to the class then we capture all categories correctly

In [56]:
predicted_labels = predict_labels(text=dataset['test']['text'][1111][:512])

In [57]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['social questions', 'trade', 'agriculture, forestry and fisheries', 'agri-foodstuffs']


In [58]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][1111])) if label == 1.0])

true labels>>> ['trade', 'agri-foodstuffs']


## we capture trade and agri-foodstuffs but mislabel social questions category and 'agriculture, forestry and fisheries'

In [59]:
predicted_labels = predict_labels(text=dataset['test']['text'][2230][:512])

In [60]:
print('predicted_labels>>>', predicted_labels)

predicted_labels>>> ['trade', 'agri-foodstuffs']


In [61]:
print('true labels>>>', [id2label[idx] for idx, label in enumerate(numbers_to_classes(dataset['test']['old_labels'][2230])) if label == 1.0])

true labels>>> ['trade', 'agri-foodstuffs']


### Based on output above we can observe that Bert returns also fully correct list of labels as well.

### Based on examples above it is clear thta BERT model learned to pick up majority of the categories but very often missed one category from the true category list.  Good part Bert picks up majority of the classes, bad it often misses one class

## Metrics that would be useful for multilabel classification are F1 score and AUC/ROC curve. We will stick to F1

In [62]:
{'eval_loss': 0.16553713381290436,
 'eval_f1': 0.8302390279906887,
 'eval_roc_auc': 0.8860434580006998,
 'eval_accuracy': 0.3152,
 'eval_runtime': 36.0575,
 'eval_samples_per_second': 138.668,
 'eval_steps_per_second': 8.681,
 'epoch': 10.0}

{'eval_loss': 0.16553713381290436,
 'eval_f1': 0.8302390279906887,
 'eval_roc_auc': 0.8860434580006998,
 'eval_accuracy': 0.3152,
 'eval_runtime': 36.0575,
 'eval_samples_per_second': 138.668,
 'eval_steps_per_second': 8.681,
 'epoch': 10.0}

## above we can observe performance measures of BERT model trained with 50 epochs for the test set with f1 score around 80%

In [63]:
1/21

0.047619047619047616

## To compare with random classifier we have 21 categories so random classifier which always predicts same category should be able to obtain around 5% (equal distribution of categoris in datasets) accuracy while our model has 31.5% on test set.